# 13/05/2024: First Simulations 

Serves the purpose to understand the problem of fairness under distribution shifts closer and to observe what happens when different (fair) models get transferred to different environments

### 0. Import of all neccesary packages 

In [99]:
import numpy as np
import pandas as pd 
import graphviz as gr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report, accuracy_score
from xgboost import XGBClassifier, DMatrix
#from scipy.stats import bernoulli
random_seed = 10


## 1. Create a dataset/ import a real world dataset

### 1.1 Synthetic data

We generate synthetic data according to the standard fairness model (SFM) introduced by Plecko and Bareinboim (2024).
We will construct two SCMs:
- 1: There is no direct effect (edge) from protected attribute X to outcome Y
- 2: There is a direct effect (edge) from protected attribute X to outcome Y


#### SCM1: no direct effect

In [223]:
np.random.seed(random_seed)
# Generate synthetic data according to the Standard-Fairness-Model (SFM) introduced by Plecko and Bareinboim (2024)
n=10000

#Define SCM generating function
def SCM(n, direct_effect, mediator_effect, effect_X_W, spurious_effect):
    # U -> exogenous variable drawn from N(0,1)
    U = np.random.normal(0,1.5, size=(n,1))
    # X -> {0,1}, representing protected attribute
    X = np.random.binomial(n=1,p=(np.exp(U)/(1+np.exp(U))), size=(n,1))
    # Z -> {0,1}, representing spurious attribute
    Z = np.random.binomial(n=1,p=(np.exp(U)/(1+np.exp(U))), size=(n,1))
    # W ->  {0,1}, representing mediating attribute
    prob_W = 0.5 + effect_X_W*X
    W = np.random.binomial(n=1,p=prob_W, size=(n,1))
    # Y -> {0,1}, representing binary outcome
    prob_Y = 0.1 + direct_effect*X + mediator_effect*W + spurious_effect*Z
    Y = np.random.binomial(n=1,p=prob_Y, size=(n,1))
    # Generate the data matrix
    synthetic_data = np.column_stack((X,W,Z,Y))
    synthetic_data_df = pd.DataFrame(synthetic_data, columns = ["X", "W","Z", "Y"])
    return synthetic_data_df


#Generate synthetic data according to SCM1:
SCM_no_direct_effect  = SCM(n, direct_effect=0, mediator_effect=0.25, effect_X_W=0.1, spurious_effect=0.45)

#Display proportion of positive outcomes for each group
print("Proportion of positive outcomes for each group:")
print(SCM_no_direct_effect.groupby("X")["Y"].mean())
print("\n")

#Display proportion of values of X
print("Proportion of X=0 and X=1:")
print(SCM_no_direct_effect["X"].value_counts(normalize=True))
print("\n")


print("The total variation of Y with respect to the binary protected attribute X:")
print(SCM_no_direct_effect.groupby("X")["Y"].mean().diff().iloc[-1])




Proportion of positive outcomes for each group:
X
0    0.390219
1    0.535292
Name: Y, dtype: float64


Proportion of X=0 and X=1:
X
1    0.5072
0    0.4928
Name: proportion, dtype: float64


The total variation of Y with respect to the binary protected attribute X:
0.14507264226309963


The corresponding directed acyclic graph (DAG) according to SCM1 can be described as follows, where U is the unobserved exogenous variable confounding X and Z.

In [ ]:
synthetic_data_DAG = gr.Digraph()   
synthetic_data_DAG.edge("X", "W")
synthetic_data_DAG.edge("W", "Y")
synthetic_data_DAG.edge("Z", "Y")
synthetic_data_DAG.edge("U", "Z", style="dotted")  
synthetic_data_DAG.edge("U", "X", style="dotted") 
synthetic_data_DAG



#### SCM2:  direct effect

In [244]:
np.random.seed(random_seed)
# Generate synthetic data according to the Standard-Fairness-Model (SFM) introduced by Plecko and Bareinboim (2024)


#Generate synthetic data according to SCM2:
SCM_direct_effect  = SCM(n, direct_effect=0.1, mediator_effect=0.25, effect_X_W=0.1, spurious_effect=0.35)

#Display proportion of positive outcomes for each group
print("Proportion of positive outcomes for each group:")
print(SCM_direct_effect.groupby("X")["Y"].mean())
print("\n")

#Display proportion of values of X
print("Proportion of X=0 and X=1:")
print(SCM_direct_effect["X"].value_counts(normalize=True))
print("\n")


print("The total variation of Y with respect to the binary protected attribute X:")
print(SCM_direct_effect.groupby("X")["Y"].mean().diff().iloc[-1])




Proportion of positive outcomes for each group:
X
0    0.347606
1    0.569795
Name: Y, dtype: float64


Proportion of X=0 and X=1:
X
1    0.5072
0    0.4928
Name: proportion, dtype: float64


The total variation of Y with respect to the binary protected attribute X:
0.22218943320086854


The corresponding DAG according to SCM2 can be described as follows, where U is the unobserved exogenous variable confounding X and Z.

In [ ]:
synthetic_data_DAG = gr.Digraph()   
synthetic_data_DAG.edge("X", "W")
synthetic_data_DAG.edge("X", "Y")
synthetic_data_DAG.edge("W", "Y")
synthetic_data_DAG.edge("Z", "Y")
synthetic_data_DAG.edge("U", "Z", style="dotted")  
synthetic_data_DAG.edge("U", "X", style="dotted") 
synthetic_data_DAG

## 2 Fit models




### 2.1 Unconstrained model 
#### 2.1.1 XGBoost

We will be using XGBoost as our classifier as it is a widely used algorithm excelling at tabular data and therefore also being used a lot in real-life applications, which might inherit some fairness violations.


#### 2.1.1.1 Fitting unconstrained model with data from SCM 1 - no direct effect

In [246]:
#one-hot encode the features X, W, Z and store as integer matrix
X_encoded = pd.get_dummies(SCM_no_direct_effect["X"], prefix="X")
W_encoded = pd.get_dummies(SCM_no_direct_effect["W"], prefix="W")
Z_encoded = pd.get_dummies(SCM_no_direct_effect["Z"], prefix="Z")
features_df = pd.concat([X_encoded, W_encoded, Z_encoded], axis=1)
features_df = features_df.astype(int) 

#add the target variable Y to get the final dataset to be passed to classifier
scm1_dataset = pd.concat([features_df, SCM_no_direct_effect["Y"]], axis=1)


#split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(scm1_dataset.drop("Y", axis=1), scm1_dataset["Y"], test_size=0.33, random_state=random_seed)


#fit the XGBoost classifier
xgb = XGBClassifier(n_estimators=2, max_depth=3, learning_rate=1, objective='binary:logistic', random_state=random_seed)
unconstrained_model_SCM1 = xgb.fit(X_train, y_train)
#obtrain predictions from unconstrained model
y_pred = unconstrained_model_SCM1.predict(X_test)

#display model specification
print(unconstrained_model_SCM1)
print("\n")

#display accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("\n")



#display total variation between predicted outcomes being positive for each group (equivalent to demographic parity with respect to predicted outcomes)
X_test_row_indices = X_test.index
features_and_predictions = SCM_no_direct_effect.iloc[X_test_row_indices].drop("Y",axis=1)
features_and_predictions["Predictions"] = y_pred
total_variation_unsconstrained = (features_and_predictions.groupby("X")["Predictions"].mean().diff().iloc[-1].round(4))
print("Total Variation/Demographic Parity of the unconstrained classifier: ", total_variation_unsconstrained )

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2, n_jobs=None,
              num_parallel_tree=None, random_state=10, ...)


Accuracy: 73.97%


Total Variation/Demographic Parity of the unconstrained classifier:  0.2933


# TO DO: COMMENT ON RESULTS

#### 2.1.1.2 Fitting unconstrained model with data from SCM 2 - direct effect

In [245]:
#one-hot encode the features X, W, Z and store as integer matrix
X_encoded = pd.get_dummies(SCM_direct_effect["X"], prefix="X")
W_encoded = pd.get_dummies(SCM_direct_effect["W"], prefix="W")
Z_encoded = pd.get_dummies(SCM_direct_effect["Z"], prefix="Z")
features_df = pd.concat([X_encoded, W_encoded, Z_encoded], axis=1)
features_df = features_df.astype(int) 

#add the target variable Y to get the final dataset to be passed to classifier
scm2_dataset = pd.concat([features_df, SCM_direct_effect["Y"]], axis=1)


#split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(scm2_dataset.drop("Y", axis=1), scm2_dataset["Y"], test_size=0.33, random_state=random_seed)


#fit the XGBoost classifier
xgb = XGBClassifier(n_estimators=2, max_depth=3, learning_rate=1, objective='binary:logistic', random_state=random_seed)
unconstrained_model_SCM2 = xgb.fit(X_train, y_train)
#obtrain predictions from unconstrained model
y_pred = unconstrained_model_SCM2.predict(X_test)

#display model specification
print(unconstrained_model_SCM2)
print("\n")

#display accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("\n")



#display total variation between predicted outcomes being positive for each group (equivalent to demographic parity with respect to predicted outcomes)
X_test_row_indices = X_test.index
features_and_predictions = SCM_direct_effect.iloc[X_test_row_indices].drop("Y",axis=1)
features_and_predictions["Predictions"] = y_pred
total_variation_unsconstrained = (features_and_predictions.groupby("X")["Predictions"].mean().diff().iloc[-1].round(4))
print("Total Variation/Demographic Parity of the unconstrained classifier: ", total_variation_unsconstrained )
print("\n")


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2, n_jobs=None,
              num_parallel_tree=None, random_state=10, ...)


Accuracy: 71.21%


Total Variation/Demographic Parity of the unconstrained classifier:  0.4773




# TO DO: COMMENT ON RESULTS

### 2.2 Model with fairness constraint
#### 2.1.1 XGBoost + Demographic Parity

We will be using XGBoost as our classifier as it is a widely used algorithm excelling at tabular data and therefore also being used a lot in real-life applications, which might inherit some fairness violations.
